
1. Définir le problème
 

2. Récupérer les données
 

3. **Analyser et nettoyer les données**
  

4. Préparer les données
  

5. Evaluer plusieurs modèles
  

6. Réglage fin des modèles
 

7. Surveiller son modèle

# 📌 3. Analyse et Nettoyage des Données  

## 🚀 Objectif
Dans cette section, nous allons **analyser et nettoyer** nos données pour nous assurer qu'elles sont exploitables avant de les utiliser dans un modèle de **Machine Learning**.  

## 🔍 Étapes du processus
1. **Importer les bibliothèques nécessaires**  
2. **Charger les données**  
3. **Supprimer les variables non contributives**  
   - Variables constantes  
   - Variables fortement corrélées  
4. **Gérer les valeurs aberrantes (outliers)**  
5. **Visualiser les données**  
6. **Catégoriser les résultats gamma (classification)**  
7. **Sauvegarder les bases de données nettoyées**  


## 🔹 3.1. Importation des bibliothèques nécessaires

In [ ]:
# Gestion des avertissements
import warnings  # Ignore certains avertissements inutiles lors de l'exécution du code


import pandas as pd  # Gestion des DataFrames
import numpy as np  # Calculs numériques
import math  # Fonctions mathématiques
import seaborn as sns  # Graphiques avancés, sert essentiellement à améliorer l'esthétique des graphiques
import matplotlib.pyplot as plt  # Affichage de graphiques
from pandas.plotting import scatter_matrix  # Matrices de nuages de points
from sklearn.feature_selection import VarianceThreshold  # Suppression de variables constantes
from scipy import stats  # Tests statistiques
from numpy import percentile  # Calcul des percentiles

warnings.filterwarnings("ignore")  # Désactiver les warnings pour ne pas surcharger l'affichage

## 🔹 3.2. Chargement des données

Nous utilisons le fichier **`DataSet_RegionPelvienne.csv`** généré précédemment.  
Nous supprimons la colonne `Unnamed: 0` qui correspond aux anciens index inutiles.


In [ ]:
# Chargement du fichier CSV
df = pd.read_csv("/content/DataSet_RegionPelvienne.csv", sep = ",", header=0)

df = df.drop(columns = ["Unnamed: 0"]) #Colonne créée avec les anciens index. On n'en a pas besoin donc on la supprime directement ici.

df.head()

## 🔹 3.3. Suppression des données non contributives  

Certaines variables **n'apportent pas d'information utile** et peuvent être supprimées :
1. **Variables constantes** : colonnes avec une variance de 0  
2. **Variables fortement corrélées** : colonnes avec informations redondantes



### ✂️ Suppression des variables constantes  



Nous utilisons la méthode **`VarianceThreshold()`** pour identifier et supprimer les variables constantes.  
Par défaut, le seuil est fixé à 0 => la méthode supprime les colonnes dont la variance est à 0 => les colonnes de données constantes. Néanmoins, on pourrait aussi souhaiter supprimer les colonnes dont les données varient très peu donc avec un seuil non nul. 


In [ ]:
# Initialisation du filtre de variance
vt = VarianceThreshold() #seuillage par défaut, pas de modification des paramètres. 

# On applique le seuillage défini dans vt à notre dataframe df avec la méthode .fit. 
_ = vt.fit(df)  # a noter : le _ permet d'accéder à la volée à la valeur du résultats précédent. Equivalent ici à vt = vt.fit(df).

# Obtention des colonnes non contributives
mask = vt.get_support() #méthode .get_support() de la classe VarianceThreshold pour obtenir un masque booléen des variables sélectionnées (True si conservées)

# Affichage des colonnes supprimées
print("Variables constantes supprimées :", list(df.loc[:, ~mask].columns)) # On affiche les noms des colonnes de données qui sont constantes (dans une liste pour faciliter l'affichage)
print("Total :", len(mask[mask == False])) # On compte le nombre de variables constantes

# Suppression des variables constantes
df = df.loc[:, mask] #On ne garde que les variables avec une variance différente de 0

# Vérification du DataFrame après suppression
df.head()


### ✂️ Suppression des variables fortement corrélées  

Les variables **très corrélées** apportent la même information et peuvent perturber l'apprentissage du modèle (plus il y a de variables plus l'entrainement sera long et le modèle risque d'apprendre plus difficilement). Elles peuvent également nuire à l'interprétation du modèle (les variables corrélées seront utilisées aléatoirement de manière équivalente par le modèle, ce qui peut diminuer l'importance de chacune d'elle et au final on peut mal interpréter l'importance de ce que décrivent ces variables pour le modèle quand on cherchera à comprendre comment il fonctionne).    
Nous utilisons le **test de normalité de Shapiro-Wilk** pour déterminer le test de corrélation adapté :


In [ ]:
# Suppression de la colonne cible pour l'analyse des corrélations entre variables
variables = df.drop(columns=['Gamma22loc10'])

# Test de Shapiro-Wilk
for cm in variables.columns: #cm pour "complexity metric"
    shapiro_test = stats.shapiro(variables[cm])
    p = shapiro_test.pvalue

    if p_value > 0.05:
        print(cm + " suit une distribution NORMALE (ne rejette pas H0).")
    else:
        print(cm + " non normale (rejette H0).")


Le test de Shapiro a tendance être plus adapté pour les taille d'échantillons moyenne (environ 50-1000 exemples). Ici à plus de 1300 échantillons, le test d'**Agostino-Pearson** aurait aussi pu être adapté : 

In [ ]:
# Test de D'Agostino-Pearson
for cm in variables.columns:
    dagostino_test = normaltest(variables[cm])
    p = dagostino_test[1]

    if p > 0.05:
        print(cm + " suit une distribution NORMALE (ne rejette pas H0).")
    else:
        print(cm + " non normale (rejette H0).")


Et celui de **Anderson-Darling** particulièrement adapté pour les grands échantillons (>1000 exemples) : 

In [ ]:
# Test d'Anderson-Darling
for cm in variables.columns:
    anderson_test = anderson(variables[cm])
    p = 0  # Initialisation de la p-valeur

    # Déterminer la p-valeur en fonction des niveaux de signification
    for i in range(len(anderson_test.significance_level)):
        if anderson_test.statistic < anderson_test.critical_values[i]:
            p = anderson_test.significance_level[i]
            break

    if p > 0.05:
        print(cm + " suit une distribution NORMALE (ne rejette pas H0).")
    else:
        print(cm + " non normale (rejette H0).")


<font size = 4> 🔥 **Matrice de corrélation** </font>    

Différents tests de corrélation peuvent être utilisés : 
- **Pearson** : variables continues et normalement distribuées
- **Spearman** et **Kendall** : variables continues ou ordinales, non nécessairement normalement distribuées.
- **ANOVA** et **Kruskal-Wallis** : variables mixtes, continues (normales pour ANOVA et non normales pour Kruskal Wallis) et catégorielles. 


Nous utilisons la **corrélation de Kendall** pour identifier les variables fortement corrélées.  
Les valeurs > **0.8** indiquent une redondance forte entre deux variables.


<div class="alert alert-block alert-danger">
<b>Attention:</b> La cellule ci-dessous ne peut être exécutée qu'une seule fois. La 2nde fois, elle affichera une erreur car les colonnes demandées ont déjà été supprimées.
</div>

In [ ]:
# Calcul de la matrice de corrélation
corr_df = variables.corr(method='kendall').abs()

# Création d'une matrice triangulaire supérieure
upper = corr_df.where(np.triu(np.ones(corr_df.shape), k=1).astype(bool))

# Index des colonnes avec un indice de corrélation >0.8
# Possibilité de modifier cette valeur pour tester
to_drop = [column for column in upper.columns if any(upper[column] > 0.8)]

print("Variables corrélées supprimées :", to_drop)
print("Total :", len(to_drop))

#Pour chaque duo de variables corrélées, on supprime aléatoirement l'une des deux variables concernées. 
df = df.drop(columns=to_drop)


In [ ]:
#Affichage d'un extrait de la matrice de corrélation
#On stocke dans la variable temp quelques unes de nos variables pour l'exemple d'affichage
temp = pd.DataFrame(variables, columns =['MUperGy',
 'AAV',
 'LSV',
 'MCS',
 'AFW',
 'ALT',
 'CAS',])
corr_temp = temp.corr(method='kendall') #calcul de la matrice de corrélation : correlation entre chaque duo de variable dans le dataframe temp
plt.figure(figsize=(8, 8)) # on crée une figure
# on crée un masque pour ne garder que la moitié de la matrice (les résultats apparaissent en double sinon, par exemple AAV vs MCS et MCS vs AAV)
# np.triu = triangle supérieur d'un tableau, toutes les données sous la diagonale du tableau sont passées à 0
# np.ones_like = retourne une matrice de 1 des mêmes dimensions que la matrice donnée en paramètres. Le dtype force le type des variables de sorties du tableau.
mask = np.triu(np.ones_like(corr_temp, dtype=bool)) # tableau de 0 (False) sous la diagonale et de 1 (True) au dessus

# Heatmap : carte des valeurs de corrélation. Les valeurs ne sont pas affichées quand le masque est "True". 
# Les autres paramètres permettent de régler l'affichage. 
heatmap = sns.heatmap(corr_temp, mask=mask, vmin=-1, vmax=1, annot=True,annot_kws={"size":10}, cmap='BrBG' , linewidth=1, linecolor='w')
heatmap.set_title('Matrice de corrélation triangulaire', fontdict={'fontsize':18}, pad=16);

>⚠️ **Le coefficient de corrélation ne mesure que les corrélations linéaires (Spearman, Pearson) ou monotones (Kendall) entre les variables.**


## 🔹 3.4 Gestion des Outliers (valeurs aberrantes des cibles)

<font size = 3> 🔍 **Détection** </font>


   - **Ecart interquartile** (IQR - Interquartile Range) : on considère qu'une valeur est un outlier si elle est en dehors de l'intervalle `[Q1 - 1.5*IQR, Q3 + 1.5*IQR]` où Q1 est le 1er quartile, Q3 le 3ème quartile et IQR = Q3 - Q1.
   - **Z-score** : on considère qu'une valeur est un outlier si sa distance en nombre d'écart-types par rapport à la moyenne est supérieure à 3.   
      $ z = \frac{x - \mu}{\sigma} $ où $x$ est la valeur de la donnée, $\mu$ la moyenne de l'ensemble de données et $\sigma$ l'écart-type de l'ensemble de donnée. Si $|z|$>3 alors on considère que la valeur est un outlier. 
    

Les **valeurs aberrantes**  des résultats gamma qui peuvent biaiser l'estimation de la limite de tolérance et également l'apprentissage du modèle.  Ces valeurs aberrantes peuvent s'expliquer par des problèmes au moment de la mesure (panne, dérive du détecteur, mauvaise position programmée, etc)    
Nous utilisons la méthode **IQR (Interquartile Range)** pour détecter et supprimer les valeurs extrêmes. Cependant, comme les valeurs gamma hors tolérance sont rares et que ce sont elles que l'on veut spécifiquement détecter, il faut faire attention à ne pas supprimer les valeurs avec un seuil trop bas. Nous avons donc choisi d'utiliser la méthode IQR mais avec un seuil à `6*IQR`. 


<div class="alert alert-block alert-danger">
<b>Attention:</b> La cellule ci-dessous ne peut être exécutée qu'une seule fois. La 2nde fois, elle n'affichera pas d'erreur mais elle va multiplier les indices gamma une seconde fois par 100 et définir un nouveau seuil de coupure. 
</div>

In [ ]:
# Conversion des indices gamma en pourcentage
df.Gamma22loc10 = df.Gamma22loc10*100

# Calcul du 1er et 3ème quartile
q25, q75 = percentile(df.Gamma22loc10, 25), percentile(df.Gamma22loc10, 75)

# Calcul de l'écart interquartile (IQR)
iqr = q75 - q25

#On affiche les résultats
print('Percentiles: 25th=%.3f, 75th=%.3f, IQR=%.3f' % (q25, q75, iqr))

# Définition d'un seuil de coupure pour détecter les outliers : on a choisi ici 6*iqr
cut_off = iqr * 6
lower, upper = q25 - cut_off, q75 + cut_off
print("cut off = ", cut_off)

#On définit les cutoff sup et inf
lower, upper = q25 - cut_off, q75 + cut_off

print("lower cut off = ", lower)

# Identification des outliers
outliers = [x for x in df.Gamma22loc10 if x < lower or x > upper]

print('outliers identifiés : %d' % len(outliers))

# suppression des outliers
outliers_removed = [x for x in df.Gamma22loc10 if x >= lower and x <= upper]

print('Observations non outliers: %d' % len(outliers_removed))

df = df[df.Gamma22loc10>= lower]

## 🔹 3.5 Visualisation des Données  

### 🔦 Corrélation des variables avec les cibles  

Nous affichons les variables **les plus corrélées** avec `Gamma22loc10`.


In [ ]:
# Calcul de la matrice de corrélation
matrice_corr = df.corr(method='kendall').abs()

# Affichage des 20 variables les plus corrélées avec Gamma22loc10
matrice_corr["Gamma22loc10"].sort_values(ascending=False).head(21)


>⚠️ **Le coefficient de corrélation ne mesure que les corrélations linéaires (Spearman, Pearson) ou monotones (Kendall) entre les variables.**





### 📊 Distribution des variables  
Nous traçons les **distributions des variables les plus importantes**.

In [ ]:
# Sélection des variables pour la visualisation
attributs = ["Gamma22loc10", "EM", "BI", "EMmin", "BIiqr", "BImax", "BIstd"]

# Matrice de nuages de points
scatter_matrix(df[attributs], figsize=(15, 8), diagonal='kde')

# Ajustement des axes
for i in range(1, 7):
    plt.gca().set_ylim(95, 100)
    plt.gca().set_xlim(95, 100)

plt.show()

## 🔹 3.6 Catégorisation des résultats PSQA  

Nous devons transformer `Gamma22loc10` en **classe binaire** pass/fail pour la classification.


### 🔧 Binarisation

<span style="color:#2980b9"> **1. On définit un seuil pour la binarisation** </span>

In [ ]:
# On définit un seuil, par exemple le 10ème percentile de la distribution de gamma
seuil = np.percentile(df.Gamma22loc10, 10)

#Affichage de la distribution
gamma = df.Gamma22loc10
plt.figure(figsize=(12,6))
g = sns.displot(gamma)
plt.xlabel("GPR", size=14)
plt.ylabel("Nombre d'exemples", size=14)
plt.axvline(x=seuil,color='red',linestyle='dashed',linewidth=1, label='Percentile')
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left', prop={'size': 12})
plt.show()

<span style="color:#2980b9"> **2. On attribue 1 aux observations sous le seuil (celles que l'on veut prédire) et 0 à celles au-dessus** </span>

<div class="alert alert-block alert-danger">
<b>Attention:</b> La cellule ci-dessous ne peut être exécutée qu'une seule fois. La 2nde fois, elle écrasera l'enregistrement de df avant binarisation et renverra une erreur car la colonne Gamma22loc10 a été supprimée
</div>

In [ ]:
# Binarisation
# Ajout d'une colonne nommée Echec dans laquelle les indices gamma <=seuil sont passés à 1 (classe positive, 'fail'), et ceux >LCLx à 0 (classe négative, 'pass')
df = df.assign(Echec=pd.cut(df.Gamma22loc10,
                               bins=[0, seuil, 100],
                               labels=[1, 0]))

# Suppression de la colonne des indices gamma
df = df.drop(columns=["Gamma22loc10"])

# Attribution du type "int" à la colonne Echec qui contient des 0 et des 1. 
df['Echec'] = df.Echec.astype('int64')


In [ ]:
#Visualisation des données
fig = df.hist(xlabelsize=5, ylabelsize=5,figsize=(13,17) )
[x.title.set_size(10) for x in fig.ravel()]
plt.show()

<font size = 4>📊 Informations que l'on peut extraire de ces histogrammes :</font>


* **Différences d'echelles** entre les variables
* **Forme des distributions** : plusieurs distributions sont fortement dissymétriques.

Ces deux paramètres peuvent nuire aux performances des modèles.


## 💾 3.6 Sauvegarde des données 

In [ ]:
# Sauvegarde du fichier CSV après binarisation
df.to_csv('DataSet_RegionPelvienne_Class.csv')

📌 **Un fichier CSV a été créé et est prêt pour les prochaines étapes du projet** 🚀

<font size = 6>⚠️</font> 

**Télécharger le fichier**